# 모듈 불러오기

In [1]:
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 개인 pc 의 chromedriver.exe 의 주소로 설정해야함
* chromedriver 설치 필요

In [ ]:
driver = wd.Chrome('C:/Users/user/chromedriver.exe')

# 리뷰 크롤링하기

In [174]:
############################ 예매 사이트로 이동 ############################
url = 'https://booking.naver.com/review/bizes/421150'
driver.get(url)
time.sleep(2)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

################################ 페이지 뽑기################################
################## 수동으로 지정할 시 for문 범위 직접 변경 #################
page = soup.find_all('span',{'ng-bind':'$ctrl.lastPage'}); 
for x in page:
    page = x.get_text()

################################## 크롤링 ##################################
# 리뷰 내용, 작성자 ID, 방문 날짜
review = []; review_id = []; review_date = []; 

for y in tqdm(range(0,int(page))):
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 리뷰 내용
    w = soup.find_all('p',{'class':'review'})
    for x in w:
        review.append(x.get_text())

    # 작성자 ID
    w = soup.find_all('span',{'class':'name'})
    for x in w:
        # 주최측에서 답변을 단 경우, 답변에 해당하는 내용은 저장X
        if '그날들' not in x.get_text():
            review_id.append(x.get_text())

    # 방문 날짜
    w = soup.find_all('span',{'ng-bind': '$ctrl.getVisitDate(item.completedDateTime || item.useDate)'}); 
    for x in w:
        review_date.append(x.get_text())
        
    # 페이지 넘기기 위해 페이지 xpath 주소 저장
    next_page = driver.find_element_by_xpath('//*[@id="container"]/review-list/div/div/div/div[2]/a[2]')
    # review 변수에 저장된 곳 클릭
    next_page.send_keys(Keys.ENTER)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:56<00:00,  8.84it/s]


In [166]:
# 제대로 뽑혔는지 확인
len(review_id);len(review); len(review_date)

900

900

900

## 부족한거 채우기

In [169]:
# 공연명, 공연 장소, 공연 기간, 공연 시간, 관람 연령, 가격, 장르
name = []; place = []; period = []; runTime = []; age = []; price = []; genre = [];

for x in range(len(review)):
    name.append('그날들')
    place.append('충무아트센터 대극장')
    period.append('2020.11.13 ~ 2021.03.07')
    runTime.append('165분')
    age.append('8세이상')
    price.append('R석 120,000원 S석 80,000원 A석 50,000원')
    genre.append('뮤지컬')

In [170]:
# 맨 처음만
a = review; 
b = review_id;
c = review_date;
d = name;
e = place;
f = period;
g = runTime;
h = age;
i = price;
j = genre;

In [137]:
# 처음 이후
a += review; 
b += review_id;
c += review_date;
d += name;
e += place;
f += period;
g += runTime;
h += age;
i += price;
j += genre;

In [177]:
# 제대로 뽑혔는지 확인
len(a); len(b); len(c); len(d); len(e); len(f); len(g); len(h); len(i); len(j)
type(a); type(b); type(c); type(d); type(e); type(f); type(g); type(h); type(i); type(j)

690

690

690

690

690

...

list

list

list

list

list

# 데이터 프레임에 저장

In [175]:
df = pd.DataFrame({ '공연명': d,
                    '공연 장소': e,
                    '공연 기간':f,
                    '공연 시간': g,
                    '관람 연령':h,
                    '가격':i,
                    '장르': j,
                    '리뷰 내용':a,
                    '예약자ID':b,
                    '방문 날짜':c})

# 엑셀 저장

In [178]:
df.to_excel('C:/Users/user/정세영/review_crawling.xlsx')